# ABoffsets_LSSTComCam_c26202


Created:  2024.11.15

## 1. Initial Setup...

### 1.1 Import useful python packages

In [ ]:
# Generic python packages
import pylab as plt
import numpy as np
import pandas as pd
import glob
import math
import os
import gc

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')

### 1.2 Set some standard variables

**Tina:  We might need to update the values for `repo` and `collections` when we find out the official values for these on Monday or so.  That said, the current values seem to be giving results...**

**UPDATE (18 Nov 2024):  These are the correct alues for `repo` and `collections`!  Yay!**

In [ ]:
# See https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.-ComCam
# and https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs
repo = 'embargo'
collections = 'LSSTComCam/runs/DRP/20241101_20241113/w_2024_46/DM-47566'

instrument = 'LSSTComCam'
skymap_name = 'lsst_cells_v1'


### 1.3 Define some useful python methods/classes for use later in the notebook

In [ ]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## 2. LSST Synthetic AB magnitudes for C26202, based on official filter LSST (v1.9) bandpasses

**Tina:  Could you update these values based on the ugrizy values for basename=c26202_mod_007 in:**

**`/home/d/dltucker/DATA/SynthMags/synthMagColorList.lsst_v1.9.calspec_20240603.added_info.csv` ?**  

**Currently, the values below are from the Dark Energy Survey (DES) ugrizy filter passbands.**

**Thanks!**



In [ ]:
# DES DR2
# c26202_mod_007 (v2024_06_03 release of STScI/HST CalSpec standards).
u_hst24 = 17.4843
g_hst24 = 16.6840
r_hst24 = 16.3319
i_hst24 = 16.2515
z_hst24 = 16.2360
y_hst24 = 16.2493

# LSST v1.9
# c26202_mod_007 (v2024_06_03 release of STScI/HST CalSpec standards).
u_hst24 = 17.5909
g_hst24 = 16.6944
r_hst24 = 16.3634
i_hst24 = 16.2619
z_hst24 = 16.2669
y_hst24 = 16.2424

# LSST v1.9
# c26202_mod_007 (v2024_06_03 release of STScI/HST CalSpec standards).
u_hst24 = 17.5728
g_hst24 = 16.6919
r_hst24 = 16.3620
i_hst24 = 16.2638
z_hst24 = 16.2451
y_hst24 = 16.2445


## 3. Query USDF Butler for ComCam measurements of C26202

### 3.1 Instantiate Butler

In [ ]:
butler = dafButler.Butler(repo, collections=collections)

### 3.2 Find all the `calexp`'s that overlap the sky position of C26202

RA, DEC of C26202 in degrees (from `/home/d/dltucker/DATA/SynthMags/synthMagColorList.lsst_v1.9.calspec_20240603.added_info.csv`):

In [ ]:
raDeg = 53.136845833333325
decDeg = -27.86349444444444

Find the `dataId`'s for all `calexp`'s in this repo/collection that overlap the RA, DEC of C26202:

In [ ]:
datasetRefs = butler.query_datasets("calexp", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

for i, ref in enumerate(datasetRefs):    
    print(i, ref.dataId)

print(f"\nFound {len(datasetRefs)} calexps")

**Tina, could use add some code here from the DP02_04b_Intermediate_Butler_Queries tutorial notebook so we can view these `calexp` images?  Maybe something from Section 3.1 from that tutorial notebook.  It would be good to take a look at the individual images in case there are any weird `calexp` images that we should ignore.**

Now, loop over the `datasetRefs` again, but this time grab the contents of the `sourceTable` table for each `ref` and combine into all into one big pandas DataFrame.  

In [ ]:
src_list = []

for i, ref in enumerate(datasetRefs):
    
    # Retrieve sourceTable for this visit & detector...
    dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
    src = butler.get('sourceTable', dataId=dataId)
    src_list.append(src)
    print(f"{i} Visit {ref.dataId['visit']}, Detector {ref.dataId['detector']}:  Retrieved catalog of {len(src)} sources.")

src_all = pd.concat(src_list, ignore_index=True)

print("")
print(f"Total combined catalog contains {len(src_all)} sources.")


Let's look at the result:

In [ ]:
src_all

Let's save `src_all` as a CSV file that we can download and examine with TOPCAT:

In [ ]:
src_all.to_csv('LSSTComCam_C26202_fields.csv', index=False)

**Tina, could you download and look at/plot the contents of `LSSTComCam_C26202_fields.csv` with TOPCAT?  Maybe sky positions, calibFlux, psfFlux, magCalib (= -2.5\*log10(calibFlux)+31.4), magPSF (= -2.5\*log10(calibFlux)+31.4) for the different filters, etc.?** 

**Remember, the sky position for the star C26202 is raDeg = 53.136845833333325, decDeg = -27.86349444444444; so maybe you can identify it (and create a TOPCAT subset for it) in TOPCAT.**

Now do the same for the `icSrc` table:

In [ ]:
icSrc_list = []

for i, ref in enumerate(datasetRefs):
    
    # Retrieve sourceTable for this visit & detector...
    dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
    icSrc = butler.get('icSrc', dataId=dataId)
    icSrc_table = icSrc.asAstropy()
    df_icSrc = icSrc_table.to_pandas()
    icSrc_list.append(df_icSrc)
    print(f"{i} Visit {ref.dataId['visit']}, Detector {ref.dataId['detector']}:  Retrieved catalog of {len(icSrc_table)} sources.")

icSrc_all = pd.concat(icSrc_list, ignore_index=True)

print("")
print(f"Total combined catalog contains {len(icSrc_all)} sources.")


Let's look at the result:

In [ ]:
icSrc = butler.get('icSrc', dataId=dataId)
icSrc_table = icSrc.asAstropy()
icSrc_table.colnames

In [ ]:
icSrc_table['id','base_CircularApertureFlux_12_0_instFlux','base_CircularApertureFlux_70_0_instFlux','base_PsfFlux_instFlux']

In [ ]:
icSrc_all

Let's save `icSrc_all` as a CSV file that we can download and examine with TOPCAT:

In [ ]:
icSrc_all.to_csv('LSSTComCam_C26202_fields_icSrc.csv', index=False)

In [ ]:
base_CircularApertureFlux_3_0_instFlux_median = np.median(icSrc_all.base_CircularApertureFlux_3_0_instFlux)
print(base_CircularApertureFlux_3_0_instFlux_median)

In [ ]:
print('3.0',np.nanmedian(icSrc_all.base_CircularApertureFlux_3_0_instFlux / icSrc_all.base_PsfFlux_instFlux))
print('4.5',np.nanmedian(icSrc_all.base_CircularApertureFlux_4_5_instFlux / icSrc_all.base_PsfFlux_instFlux))
print('6.0',np.nanmedian(icSrc_all.base_CircularApertureFlux_6_0_instFlux / icSrc_all.base_PsfFlux_instFlux))
print('9.0',np.nanmedian(icSrc_all.base_CircularApertureFlux_9_0_instFlux / icSrc_all.base_PsfFlux_instFlux))
print('12.0',np.nanmedian(icSrc_all.base_CircularApertureFlux_12_0_instFlux / icSrc_all.base_PsfFlux_instFlux))
print('17.0',np.nanmedian(icSrc_all.base_CircularApertureFlux_17_0_instFlux / icSrc_all.base_PsfFlux_instFlux))
print('25.0',np.nanmedian(icSrc_all.base_CircularApertureFlux_25_0_instFlux / icSrc_all.base_PsfFlux_instFlux))
print('35.0',np.nanmedian(icSrc_all.base_CircularApertureFlux_35_0_instFlux / icSrc_all.base_PsfFlux_instFlux))
print('50.0',np.nanmedian(icSrc_all.base_CircularApertureFlux_50_0_instFlux / icSrc_all.base_PsfFlux_instFlux))
print('70.0',np.nanmedian(icSrc_all.base_CircularApertureFlux_70_0_instFlux / icSrc_all.base_PsfFlux_instFlux))


In [ ]:
icExp = butler.get('icExp', dataId=dataId)

In [ ]:
icExp_info = icExp.getInfo()

In [ ]:
print(icExp_info.getMetadata())

In [ ]:
print(dataId)

In [ ]:
%pwd

In [ ]:
%ls -lh LSSTComCam*

In [ ]:
datasetRefs = butler.query_datasets("calexp", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})


In [ ]:
for dt in sorted(butler.registry.queryDatasetTypes('*src*')):
    print(dt)

In [ ]:
for dt in sorted(butler.registry.queryDatasetTypes('*icSrc*')):
    print(dt)

In [ ]:
icSrc = butler.get('icSrc', dataId=dataId)

In [ ]:
icSrc.asAstropy()

In [ ]:
src1 = butler.get('src', dataId=dataId)

In [ ]:
src1.asAstropy()

**Let's stop here for now:**

In [ ]:
raise StopExecution

In [ ]:
# Observed DES DR2 magnitudes for C26202 (from Y6A2_COADD_OBJECTS_SUMMARY table)...
g_wavg,r_wavg,i_wavg,z_wavg,Y_wavg = 16.695784,16.342682,16.25813,16.240156,16.263472
g_auto,r_auto,i_auto,z_auto,Y_auto = 16.693335,16.341341,16.256319,16.238766,16.268082

## 4. Measure differences between the Observed ComCam and the LSST Synthetic Mags for C26202

In [ ]:
# DES DR2 AB offsets based on c26202_stisnic_007.fits from William Wester's DES-doc#15451...

print "AB offsets based on c26202_stisnic_007.fits"
print "==========================================="
print 

aboffset_i = i_wavg - i_ww
aboffset_gr = (g_wavg-r_wavg) - (g_ww-r_ww)
aboffset_ri = (r_wavg-i_wavg) - (r_ww-i_ww)
aboffset_iz = (i_wavg-z_wavg) - (i_ww-z_ww)
aboffset_zY = (z_wavg-Y_wavg) - (z_ww-Y_ww)

print "WAVG offsets"
print "------------"
print """i:    %10.4f""" % (aboffset_i)
print """g-r:  %10.4f""" % (aboffset_gr)
print """r-i:  %10.4f""" % (aboffset_ri)
print """i-z:  %10.4f""" % (aboffset_iz)
print """z-Y:  %10.4f""" % (aboffset_zY)
print 

aboffset_i = i_wavg - i_ww
aboffset_gr = (g_auto-r_auto) - (g_ww-r_ww)
aboffset_ri = (r_auto-i_auto) - (r_ww-i_ww)
aboffset_iz = (i_auto-z_auto) - (i_ww-z_ww)
aboffset_zY = (z_auto-Y_auto) - (z_ww-Y_ww)

print "MAG_AUTO offsets"
print "----------------"
print """i:    %10.4f""" % (aboffset_i)
print """g-r:  %10.4f""" % (aboffset_gr)
print """r-i:  %10.4f""" % (aboffset_ri)
print """i-z:  %10.4f""" % (aboffset_iz)
print """z-Y:  %10.4f""" % (aboffset_zY)
print 


## 5. Sandbox

In [ ]:
datasetRefs = butler.query_datasets("visitSummary", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

#print(datasetRefs)

print(f"\nFound {len(datasetRefs)} calexps")

In [ ]:
# Retrieve sourceTable for this visit & detector...

datasetType = 'sourceTable'
#dataId = {'visit': visit, 'detector': detector}
dataId = ref.dataId['visit']
dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
print(dataId)

src = butler.get(datasetType, dataId=dataId)

print(f"Retrieved catalog of {len(src)} sources.")

In [ ]:
butler.registry.queryDataIds(dimensions=('exposure'))